# 1. 패키지 설치 및 라이브러리 불러오기

In [ ]:
!pip install gcsfs==2023.10.0 datasets==2.17.0

In [ ]:
!pip install -qU transformers==4.38.0 accelerate==0.27.1 bitsandbytes==0.42.0 peft==0.8.2 trl==0.7.10

In [ ]:
import torch
import pandas as pd
import numpy as np
import warnings
import json
import time

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

from huggingface_hub import notebook_login

# 2. 데이터셋 로드

In [ ]:
from datasets import load_dataset

# 데이터셋 로드
dataset = load_dataset("ChuGyouk/argilla-distilabel-math-preference-dpo-korean")
# 데이터셋의 구조 확인
print(dataset)

# 3. 데이터 전처리 (프롬프트)

In [ ]:
# 'prompt' 필드 생성 함수
def format_instruction_ko(example):
    # input 없이 instruction_ko와 chosen_response_ko만 사용
    text = f"""user\n{example["instruction_ko"]}\nmodel\n{example["chosen_response_ko"]}"""
    return {'prompt': text}

# 데이터셋의 prompt 필드를 업데이트
dataset = dataset.map(format_instruction_ko)


In [ ]:
notebook_login()

# 4. 모델 로드 및 양자화 설정

In [ ]:
model_id = "google/gemma-2b"

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16)


model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0})

tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

# 4. 데이터셋 토크나이징 및 학습/검증 세트 분할

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
dataset = dataset['train'].train_test_split(test_size=0.2)
dataset

In [ ]:
train_data = dataset["train"]
test_data = dataset["test"]

# 예측 모델을 위한 함수 정의

In [ ]:
def get_completion(query: str, model, tokenizer):

  prompt_template = """user
  {query}

  model
  """
  prompt = prompt_template.format(query=query)
  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
  model_inputs = encodeds.to("cuda:0")
  generated_ids = model.generate(**model_inputs, max_new_tokens=256)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return decoded

# Fine tuning 이전
result = get_completion(query="식을 단순화하세요: 2(x + y) - 3(2x - y). 깊게 숨을 들이쉬고, 단계별로 생각하여 정확한 답변을 제공하세요.", model=model, tokenizer=tokenizer)
print(result)


# Fine-Tunning 진행

In [ ]:
torch.cuda.empty_cache()

lora_config = LoraConfig(
    r=32,
    target_modules=['o_proj', 'q_proj', 'up_proj', 'v_proj', 'k_proj', 'down_proj', 'gate_proj'],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=1000,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit",
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

trainer.train()

# 모델 예측 확인

In [ ]:
# Fine tuning 이후
result = get_completion(query="식을 단순화하세요: 2(x + y) - 3(2x - y). 깊게 숨을 들이쉬고, 단계별로 생각하여 정확한 답변을 제공하세요.",
                        model=trainer.model,
                        tokenizer=tokenizer)
print(result)

# 모델 저장

In [ ]:
new_model = "gemma-2b-math-korean-finetuned"
trainer.model.save_pretrained(new_model)